In [1]:
import os
import sys
from pathlib import Path
PATH = os.path.join(os.getcwd(), '..', '..')
sys.path.append(PATH)

import torch
import numpy as np
import matplotlib.pyplot as plt

from fastai.vision import *

from src.model_utils.cyclegan import CycleGAN
from src.model_utils.databunch import ImageTupleList
from src.model_utils.callbacks import CycleGANTrainer
from src.model_utils.loss import CycleGanLoss

In [2]:
import os
from pathlib import Path
data_dir = Path('../../data/test')

In [3]:
data = (ImageTupleList.from_folders(data_dir, 'trainA', 'trainB')
                          .split_none()
                          .label_empty()
                          .transform(get_transforms(), size = 56)
                          .databunch(bs = 4))
cycle_gan = CycleGAN(ch_in = 3, ch_out = 3, 
                     disc_layers = 3, 
                     gen_blocks = 5)
learn = Learner(data, cycle_gan, 
                loss_func = CycleGanLoss(cycle_gan), 
                opt_func = partial(optim.Adam, betas = (0.5, 0.99)),
                callback_fns = [CycleGANTrainer])

In [7]:
learn.fit(2)

epoch,train_loss,valid_loss,id_loss,gen_loss,cyc_loss,D_A_loss,D_B_loss,time
0,9.864865,#na#,1.435390,4.730968,3.698507,2.063473,4.498824,00:15


> /userhome/34/h3509807/fastai/CycleGAN-MultiMNIST/src/model_utils/callbacks.py(74)on_epoch_end()
-> return add_metrics(last_metrics,
(Pdb) continue
> /userhome/34/h3509807/fastai/CycleGAN-MultiMNIST/src/model_utils/callbacks.py(74)on_epoch_end()
-> return add_metrics(last_metrics,
(Pdb) last_metrics
[None]
(Pdb) [s for s in self.id_smter]
*** TypeError: 'SmoothenValue' object is not iterable
(Pdb) add_metrics
<function add_metrics at 0x7f374c953d90>
(Pdb) from inspect import getsource
(Pdb) print(getsource(add_metrics))
def add_metrics(last_metrics:Collection[Rank0Tensor], mets:Union[Rank0Tensor, Collection[Rank0Tensor]]):
    "Return a dictionary for updating `last_metrics` with `mets`."
    last_metrics,mets = listify(last_metrics),listify(mets)
    return {'last_metrics': last_metrics + mets}

(Pdb) None + [1,34]
*** TypeError: unsupported operand type(s) for +: 'NoneType' and 'list'
(Pdb) l
 69  	        # freeze discrimintors and unfreeze generators for generators update
 70  	  

BdbQuit: 